<a id='root'></a>
# PyUltraLight 2 Examples Notebook

(3 May 2021)

* Y. Wang: [yourong.f.wang@auckland.ac.nz](mailto:yourong.f.wang@auckland.ac.nz)
* R. Easther
***



<a id='init'></a>
# Initialization and Program Settings

## Loading Packages

In [ ]:
###### Do not touch
MinVersion = 23

import PyUltraLight2 as PyUL

if (PyUL.S_version < MinVersion):
    raise RuntimeError("You need the latest PyULN!")

PyUL.PyULCredits()

import numpy as np

np.set_printoptions(suppress=True)

import math

import numba
import numexpr as ne
import time
import pyfftw
import os
import sys
import multiprocessing
import numpy

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec
import matplotlib.animation
from matplotlib.offsetbox import AnchoredText

import scipy.fft

import h5py
import IPython
from IPython.core.display import clear_output, display, Video

from numpy import sqrt, exp, log, log10, sin, cos, tan, pi

%reload_ext autoreload
%autoreload 2

# Useful Aux Functions
ToCode = PyUL.convert
ToPhys = PyUL.convert_back
CB = PyUL.convert_between
printU = PyUL.printU

[**<font color=blue>Back to the Top</font>**](#root)
<a id='demos'></a>
# All Available Pre-Calculated Scenarios


<a id='demo_elli'></a>
## Example 1: 2Body Elliptical Orbit

In [ ]:
resol = 128

length = 10  # 1 code unit is ~38 kpc x (1e-22/m_a)^0.5
duration = 5  #1 code unit is ~70 Gyr (independent of axion mass assumption)

step_factor = 1

length_units = ''
duration_units = ''
s_mass_unit = ''
s_position_unit = ''
s_velocity_unit = ''
m_mass_unit = ''
m_position_unit = ''
m_velocity_unit = ''

m1 = 9  # User Input
Ratio1 = 1

m2 = 9  # User Input
Ratio2 = 0

Aphelion = 2.4  # User Input

e = 0.3  # Eccentricity. 0 -> Circle. 1 -> Parabola.

#================================================================
# Do not touch

M = m1 + m2
xC = Aphelion

## Simulation always starts from Aphelion, in Centre-of-mass Frame

x2 = xC * m1 / M  # Bigger Mass, Smaller Initial Position
x1 = xC - x2

a1 = x1 / (1 + e)  # Semimajor Axis from Aphelion
a2 = x2 / (1 + e)

xPeri1 = a1 * (1 - e)
xPeri2 = a2 * (1 - e)

## Initial Velocity is purely tangential.

yDot1 = np.sqrt(m2 * x1 / xC**2 * (1 - e) / (1 + e))
yDot2 = np.sqrt(m1 * x2 / xC**2 * (1 - e) / (1 + e))

particles = []
solitons = []

Halo1 = [m1, [x1, 0, 0], [0, yDot1, 0], Ratio1, 0]
Halo2 = [m2, [-x2, 0, 0], [0, -yDot2, 0], Ratio2, 0]

embeds = [Halo1, Halo2]

########### MODIFIER PARAMETERS
# Removes all ULDM Structures defined elsewhere in the IV and replaces them with a uniform wavefunction
# with given probability amplitude (code unit).

Uniform = True
Density = 0

if not Uniform:
    print('The List of Soliton Conditions is:')
    print(solitons)

print('The List of BH Conditions is:')
print(particles)

print('The List of Embedded Conditions is:')
print(embeds)

PyUL.SolitonSizeEstimate(m2, length, resol, s_mass_unit, length_units, True)

[**<font color=green>Go to Execution</font>**](#run)
<a id='demo_scatter'></a>
## Example 2: 2Body Scattering

In [ ]:
resol = 384

length_units = 'kpc' 
duration_units = 'Myr' 
s_mass_unit = 'M_solar_masses'
s_position_unit = 'kpc' 
s_velocity_unit = 'km/s' 

length = 6 #
duration = 25 #

step_factor = 1 # Large number to save sanity.

m1 = 3500 # User Input
m2 = 2400 # User Input


PyUL.SolitonSizeEstimate(m1,length,resol,s_mass_unit,length_units,True)
PyUL.SolitonSizeEstimate(m2,length,resol,s_mass_unit,length_units,True)

print(f"The (code) density is {PyUL.DensityEstimator(length,length_units,m1+m2,s_mass_unit)}")

Ratio1 = 0
Ratio2 = 0

Phase = 0 # Multiples of pi

b = 0.31 # Impact Parameter

vRel0 = 60 # Initial Speed along Collision Axis

Separation = 1.5

Uniform = False # if True, then there won't be solitons.
Density = 0
UVel = [0,0,0] # Uses s_velocity_units
density_unit = '' # Accepted units: 'MSol/pc3', 'kg/m3', and '' for dimensionless units

m_mass_unit = s_mass_unit
m_position_unit = s_position_unit
m_velocity_unit = s_velocity_unit

#================================================================
# Do not touch

# Unit 1
y1 = Separation*m2/(m1+m2)
x1 = b*m2/(m1+m2)
yDot1 = vRel0*m2/(m1+m2)


# Unit 2
y2 = Separation*m1/(m1+m2)
x2 = b*m1/(m1+m2)
yDot2 = vRel0*m1/(m1+m2)


particles = []
solitons = []
embeds = [] 

Composite1 = [m1,[x1,y1,0],[0,-yDot1,0],Ratio1,Phase*np.pi/2]
Composite2 = [m2,[-x2,-y2,0],[0,yDot2,0],Ratio2,-Phase*np.pi/2]

embeds.append(Composite1)
embeds.append(Composite2)

print(PyUL.DensityEstimator(length,length_units,(m1+m2),s_mass_unit))

[**<font color=green>Go to Execution</font>**](#run)
<a id='demo_tya'></a>
## Example 3: 4Body Tangyuan

In [ ]:
resol = 128
length = 12# 1 code unit is ~38 kpc x (1e-22/m_a)^0.5
duration = 5 #1 code unit is ~70 Gyr (independent of axion mass assumption)

step_factor = 1

length_units = '' 
duration_units = '' 
s_mass_unit = ''
s_position_unit = '' 
s_velocity_unit = '' 
m_mass_unit = ''
m_position_unit = ''
m_velocity_unit = ''

# Density, if Uniform Background Flag is Set to 'True'
# 1 Code Density Unit is equal the universe's critical matter density, 2.67e-27 kg/m^3.
density_unit = '' # Accepted units: 'MSol/pc3', 'kg/m3', and '' for dimensionless units

r = 1.4
m = 8

Ratio = 0.05

v = np.sqrt(m/r*(1+2*np.sqrt(2)))/2

vM = 1 # Change Magn. 

#================================================================
# Do not touch

v = vM * v
print("The initial tangential speed is %.4f \n" % (v))

Halo1 = [m,[r,0,0],[0,v,0],Ratio,0]
Halo2 = [m,[0,r,0],[-v,0,0],Ratio,0]
Halo3 = [m,[-r,0,0],[0,-v,0],Ratio,0]
Halo4 = [m,[0,-r,0],[v,0,0],Ratio,0]

embeds = [Halo1, Halo2, Halo3, Halo4]
#Soliton parameters are mass, position, velocity and phase (radians)

solitons = []
#particles = [[6,[-3,-3,0],[5,5,0]]]
particles = []

Uniform = False
Density = 0.001
UVel = [0,0,0] # Uses s_velocity_units

if Uniform:
    embeds = []
    BH1 = [m,[r,0,0],[0,v,0]]
    BH2 = [m,[0,r,0],[-v,0,0]]
    BH3 = [m,[-r,0,0],[0,-v,0]]
    BH4 = [m,[0,-r,0],[v,0,0]]
    particles = [BH1, BH2, BH3, BH4]


[**<font color=green>Go to Execution</font>**](#run)
<a id='demo_QM'></a>
## Example 4: Merger From Rest

In [ ]:
resol = 128

length = 4 # 1 code unit is ~38 kpc x (1e-22/m_a)^0.5
duration = 20 #1 code unit is ~70 Gyr (independent of axion mass assumption)

SimL = length/2  # Simulation Starting Region

step_factor = 1

NEmbed = 8

RandomSeed = 303022

###
np.random.seed(RandomSeed)

length_units = 'kpc' 
duration_units = 'Myr' 
s_mass_unit = 'M_solar_masses'
s_position_unit = 'kpc' 
s_velocity_unit = 'km/s' 

m_mass_unit = 'M_solar_masses'
m_position_unit = 'kpc' 
m_velocity_unit = 'km/s' 

MassMax = 4000 #
MassMin = 3000

RatioMax = 0.
RatioMin = 0.


#================================================================
# Do not touch

embeds = []
particles = []

for i in range(NEmbed):
    Location = ([(np.random.random()-1/2)*SimL/2,(np.random.random()-1/2)*SimL/2,0])
    
    Speed = 200*np.random.random()
    
    Angle = 2*np.pi*np.random.random()
    
    Velocity = [Speed*np.cos(Angle),Speed*np.sin(Angle),0]
    
    Ratio = RatioMax
    
    Mass = MassMin + np.random.random()*(MassMax-MassMin)
    
    Phase = np.pi*(np.random.random())/2
    
    embeds.append([Mass,Location,Velocity,Ratio,Phase])
    
solitons = []

Uniform = False
Density = 0
UVel = [0,0,0] # Uses s_velocity_units
density_unit = '' # Accepted units: 'MSol/pc3', 'kg/m3', and '' for dimensionless units

print(PyUL.DensityEstimator(length,length_units,(m1+m2),s_mass_unit))

[**<font color=green>Go to Execution</font>**](#run)
<a id='demo_QM'></a>
## Example 5: Dwarf Galaxies

In [ ]:
resol = 128

length = 16# 1 code unit is ~38 kpc x (1e-22/m_a)^0.5
duration = 10 #1 code unit is ~70 Gyr (independent of axion mass assumption)

step_factor = 1

NStar = 10 # Test stars per galaxy

RIn = 1
ROut = 2.3
RBaseline = 20 # Smaller galaxies are denser in stars.
MaxStarMass = 0

RandomSeed = 950209

###
np.random.seed(RandomSeed)

length_units = '' 
duration_units = '' 
s_mass_unit = ''
s_position_unit = '' 
s_velocity_unit = '' 
m_mass_unit = ''
m_position_unit = ''
m_velocity_unit = ''



embeds = [[5,[0,-3,0],[-0.03,0.375,0],0,np.pi],[7,[0,2,0],[0.02,-0.25,0],0,0]]

particles = []

def FOSR(m,r):
    return np.sqrt(m/r)

#================================================================
# Do not touch

for j in range(len(embeds)):
    
    Galaxy = embeds[j]
    
    GPos = np.array(Galaxy[1])
    GVel = np.array(Galaxy[2])
    GMass = Galaxy[0]
    
    RFac = GMass/20
    
    NStarL = np.random.randint(-2, 4)
    for i in range(NStar+NStarL):

        r = (np.random.random()*(ROut-RIn) + RIn)*GMass/RBaseline
        
        theta = 2*np.pi*np.random.random()
        v = FOSR(GMass,r)

        Position = np.array([r*np.cos(theta),r*np.sin(theta),0]) + GPos
        Velocity = np.array([v*np.sin(theta),-v*np.cos(theta),0]) + GVel
        
        Mass = MaxStarMass*np.random.random()
        
        particles.append([Mass,Position.tolist(),Velocity.tolist()])


solitons = []

Uniform = False
Density = 0
UVel = [0,0,0] # Uses s_velocity_units
density_unit = '' # Accepted units: 'MSol/pc3', 'kg/m3', and '' for dimensionless units
#print(*embeds, sep = "\n")

#print(*particles, sep = "\n")


[**<font color=green>Go to Execution</font>**](#run)
<a id='demo_byo'></a>
## [Dynamical Friction with Uniform Background]

In [ ]:
######## USER PARAMETERS BELOW ################################

speed, s_velocity_unit = (100,'km/s')

QuantumWind = True # Galileo Switch

Automatic = True

AutoMult = 1/4 # For how long should you run?

BHSpeedMultiplier = 0 # 1 = Comoving, -1 = Against, 0 = Stationary 

length_units = 'kpc'

duration = 90 #1 code unit is ~70 Gyr (independent of axion mass assumption)

duration_units = 'Myr'  # Accepted units: 's', 'yr', 'kyr', 'Myr', and '' for dimensionless units

NdB = 8  # Number of deBroglie wavelengths that fit in the simulation
rPB = 16 # Resolution per full matter wavelength.

mBH1, m_mass_unit = 10 , 'M_solar_masses'

Uniform = True
Density = 1e7 # 1 Code unit is 0.31*Critical Density
density_unit = 'Crit'


######## AUTOMATIC PROCESSING BELOW ################################

# de Broglie in Lab Frame is the Quantum Wind wavelength

if speed == 0:
    lbasic = 1
else:
    lbasic = PyUL.convert_back(np.pi*2/PyUL.convert(speed,s_velocity_unit,'v'),length_units,'l')

resol = NdB*rPB

print(f'Quantum de Broglie wavelength: {lbasic:.4g} {length_units}')

length = NdB*lbasic # Can be changed to fit more wavelengths.

MaxDuration = CB((CB(length,length_units,'m','l')/CB(speed,s_velocity_unit,'m/s','v')),'s',duration_units,'t')
print(f'Maximum allowed duration: {MaxDuration:.1f} {duration_units}')

if Automatic:
    duration = MaxDuration * AutoMult

print(f'Simulation domain edge length: {length:.4g} {length_units}')


#Density = PyUL.DensityEstimator(length,length_units,(TotalMass),s_mass_unit)/1000
##================================================================
## Other Units
# Set units for soliton parameters
    
s_position_unit = length_units
m_position_unit = length_units
m_velocity_unit = s_velocity_unit

s_mass_unit = ''

# 1 Code Density Unit is equal the universe's critical matter density, 2.67e-27 kg/m^3.
 # Accepted units: 'MSol/pc3', 'kg/m3', and '' for dimensionless units

##================================================================

if QuantumWind:
    
    
    particles = [[mBH1,[0,length/4,0],[0,-speed*BHSpeedMultiplier,0]]]
    #particles = [[mBH1,[0,0,0],[0,-speed*BHSpeedMultiplier,0]]]
    
    solitons = []

    embeds = []

    UVel = [0,-speed,0] 
    
else:

    #particles = [[mBH1,[0,-length/4,0],[0,speed,0]]]
    #particles = [[mBH1,[0,0,0],[0,speed,0]]]

    solitons = []

    embeds = []

    UVel = [0,0,0]
    
    
speedSI = -CB(speed,s_velocity_unit,'m/s','v')
mBHSI = CB(mBH1,'M_solar_masses','kg','m')

ldbM = CB(lbasic,length_units,'m','l')

beta = 2*np.pi*PyUL.G*mBHSI/speedSI**2/(ldbM)

print('Beta:', beta)

[**<font color=green>Go to Execution</font>**](#run)
<a id='demo_byo'></a>
# Build Your Own


## How to create new objects.

**Particle Config:** 
```python
[mass,[position],[velocity]]
```

**Soliton Config:**
```python
[mass,[position],[velocity],phase]
```

**Embedded Soliton Config:**
```python
[total_mass,[position],[velocity],mBH/total_mass,phase]
```

**Uniform Wavefront Speed (Previously QuantumWind):**

```python
UVel
```


**NOTE**: Embedded solitons and Uniform backgrounds inherit the soliton units setting.

In [ ]:
resol = 256
# Space Settings
length = 15 # 1 code unit is ~38 kpc x (1e-22/m_a)^0.5
length_units = 'kpc'  # Accepted units: 'm', 'km', 'pc', 'kpc', 'Mpc', 'ly', and '' for dimensionless units.

# Time Settings
duration = 8e-6 #1 code unit is ~70 Gyr (independent of axion mass assumption)
duration_units = ''  # Accepted units: 's', 'yr', 'kyr', 'Myr', and '' for dimensionless units

step_factor = 1

# Recommended Save Number Cap: 120
# Depending on resolution, this could require significant disk space.

##================================================================
## Other Flags
# Set units for soliton parameters
s_mass_unit = ''     #Accepted units: 'kg', 'solar_masses', 'M_solar_masses', and '' for dimensionless units
s_position_unit = '' #Accepted units: 'm', 'km', 'pc', 'kpc', 'Mpc', 'ly', and '' for dimensionless units
s_velocity_unit = '' #Accepted units: 'm/s', 'km/s', 'km/h', 'c', and '' for dimensionless units

# Rules ditto.
m_mass_unit = 'M_solar_masses'
m_position_unit = 'kpc'
m_velocity_unit = 'm/s'

# Density, if Uniform Background Flag is Set to 'True'
# 1 Code Density Unit is equal the universe's critical matter density, 2.67e-27 kg/m^3.
density_unit = '' # Accepted units: 'MSol/pc3', 'kg/m3', and '' for dimensionless units

##================================================================

mBH1 = 1

BH1 = [mBH1,[0,-0.3,0],[0,9e4,0]]

particles = [BH1]

soliton1=[2000,[0,0,0],[0,0,0],-0]

solitons = [soliton1]

embeds = []

########### MODIFIER PARAMETERS

# Removes all ULDM Structures defined elsewhere in the IV and replaces them with a uniform wavefunction 
# with given probability amplitude (code unit).

Uniform = False
Density = 4e10
UVel = [0,0,0] # Uses s_velocity_units
#Density = 0


if not Uniform:
    print('The List of Soliton Conditions is:')
    print(solitons)
    print('The List of Embedded Soliton Conditions is:')  
    print(embeds)
    
print('The List of BH Conditions is:')  
print(particles)



[**Back to the Top**](#root)
<a id='run'></a>
# Compiling Init File

This creates a timestamped folder using the current settings, and the integrator stores all requested files in it too.

## Grid Size and Plummer Radius

In [ ]:
gridspace = PyUL.MeshSpacing(resol, length, length_units, silent=False)
# PLUMMER RADIUS (IN LENGTH UNITS)
rP = gridspace / 2

a = PyUL.GenPlummer(rP, length_units)

## Initial Condition Sketch

In [ ]:
fig,ax = PyUL.VizInit2D(length,length_units,resol,embeds,
              solitons,s_position_unit, s_mass_unit,
              particles,m_position_unit, Uniform, Density, UVel, rP, VScale = 1)

## <font color=orange>Save Settings</font>


*Your options text should not contain commas. Please separate options with a space. No sorting needed*

**Please Always Include**

    Energy NBody DF

**To save the entire grid (not recommended)**

    3Wfn
    
**To save ULDM densities**

    3Density 2Density 1Density
    
**To save ULDM gravitational fields**

    3Grav 2Grav 1Grav
    
**To save total gravitational fields**

    3GravF 2GravF 1GravF
    
**To save phase in the x-y plane**

    2Phase

**Bonus features**

    Entropy 
(saves the sum of ρlogρ)
    
    AngMomentum
(with respect to origin, time-consuming)
    
    Momentum
(very time-consuming)
    
    

***Storage Saver ***

    Save_Options = "Minimum"
    
This will be automatically translated to 
    
    Energy 1Density NBody DF Entropy

In [ ]:
start_time = 0. # For solitons only: Pre-evolve the wavefunction phase.

NS = 32

save_format = 'npy' # npy, npz, hdf5

step_factor = 1

save_number = -1


PyUL.DispN(duration,duration_units,length,length_units,resol,step_factor,save_number)


# You can also set it to 'Minimum' to only save all 1D mesh Data.
Save_Options = '2Density 2GravF 2Phase Energy 1Density NBody 1Grav DF Entropy 1GravF'

#This Boolean List is for program's internal use
save_options = PyUL.SaveOptionsDigest(Save_Options)

In [ ]:
save_path = 'Examples'

PyUL.DSManagement(save_path, Force = False)

In [ ]:
Name = ''

Initpath = ''

run_folder = PyUL.GenerateConfig(NS, length, length_units,
                                 resol, duration, duration_units, step_factor,
                                 save_number, Save_Options, save_path,
                                 save_format, s_mass_unit, s_position_unit,
                                 s_velocity_unit, solitons,start_time,
                                 m_mass_unit, m_position_unit, m_velocity_unit,
                                 particles,embeds, Uniform,Density,density_unit,
                                 a,UVel,False,Name)


printU('Please use the following paths when executing simulation:\n', 'Export')


print(f'PyUL.evolve(\'{save_path}\',\'{run_folder}\',[SETTINGS])')
       
